数字识别的数据集来源于MNIST数据集，即美国国家标准与技术研究所（NIST）的训练集，该训练集由来自 250 个不同人手写的数字构成，其中 50% 是高中学生，50% 来自人口普查局（the Census Bureau）的工作人员。该数据集并未使用普通的图片格式存储手写数字格式，若要在tensorflow中引入该训练集，可直接运行以下代码：

In [ ]:
from tensorflow.keras.datasets import mnist
# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

若需要自制数据集，则原始数据符合一定的要求，如样本（即输入特征）的格式和命名，都有对应的标签等等。
目录`mnist\`中包括两个文件夹和两个txt文件，分别对应手写图片和标签文件。

文件夹中的图手写数字图片，命名方式为**编号_标签**，比如**0_5**代表标签为5的手写数字图片中的第0个（也就是第一个）。

txt文本中的数据如下，包含2列数据，第一列是图片名，第二列是对应的标签。

这些图片都是28*28大小的灰度图，命名方式相同，且训练集和测试集中没有重复训练集和测试集中的图片，通过自定义函数调用txt文本中的图片并将对应的标签一起保存到数据集文件中（.npy格式），实现图片名-标签一一对应到图片-标签一一对应的转换。

其简单思路为：读取一行txt数据，得到图片名+标签，根据文已知文件夹地址和图片名导入图片，绑定图片和标签（保存到两个列表的同一位置），保存。

实现代码如下：
（由于涉及文件路径的问题，建议直接运行`generate.py`程序）

In [ ]:
# 第一步，导入相关模块，增加了几个调用
import tensorflow as tf
from PIL import Image
import numpy as np
import os

# 第二步，根据输入特征和标签，自制数据集
# 2.1 导入数据集，添加数据集特征和标签路径，以及保存的路径
train_path = './mnist_image_label/mnist_train_jpg_60000/'    # 训练集输入特征路径
train_txt = './mnist_image_label/mnist_train_jpg_60000.txt'  # 训练集标签txt文件
x_train_savepath = './mnist_image_label/mnist_x_train.npy'   # 训练集输入特征存储文件
y_train_savepath = './mnist_image_label/mnist_y_train.npy'   # 训练集标签存储文件

test_path = './mnist_image_label/mnist_test_jpg_10000/'      # 测试集输入特征路径
test_txt = './mnist_image_label/mnist_test_jpg_10000.txt'    # 测试集标签文件
x_test_savepath = './mnist_image_label/mnist_x_test.npy'     # 测试集输入特征存储文件
y_test_savepath = './mnist_image_label/mnist_y_test.npy'     # 测试集标签存储文件

# 2.2 自制数据集函数
def generateds(path, txt):      # 通过函数导入数据路径和
    f = open(txt, 'r')          # 以只读形式打开txt文件
    contents = f.readlines()    # 读取文件中所有行
    f.close()                   # 关闭txt文件
    x, y_ = [], []              # 建立空列表
    for content in contents:    # 逐行取出
        value = content.split()           # 以空格分开保存到value中，图片路径为value[0] , 标签为value[1] , 存入列表
        img_path = path + value[0]        # 拼接图片路径和文件名
        img = Image.open(img_path)        # 读入图片
        img = np.array(img.convert('L'))  # 图片变为8位宽灰度值的np.array格式
        img = img / 255.                  # 数据归一化（实现预处理）
        x.append(img)                     # 归一化后的数据，贴到列表x
        y_.append(value[1])               # 标签贴到列表y_
        print('loading : ' + content)     # 打印状态提示

    x = np.array(x)           # x变为np.array格式
    y_ = np.array(y_)         # y变为np.array格式
    y_ = y_.astype(np.int64)  # y变为64位整型
    return x, y_              # 返回输入特征x，返回标签y_

# 2.3 判断数据集是否存在，是则直接导入，否则调用函数创造数据集
if os.path.exists(x_train_savepath) and os.path.exists(y_train_savepath) and os.path.exists(
        x_test_savepath) and os.path.exists(y_test_savepath):  # 判断训练集和测试集是否存在，是则直接读取
    print('-------------Load Datasets-----------------')
    x_train_save = np.load(x_train_savepath)
    y_train = np.load(y_train_savepath)
    x_test_save = np.load(x_test_savepath)
    y_test = np.load(y_test_savepath)
    x_train = np.reshape(x_train_save, (len(x_train_save), 28, 28))  # 将输入特征转换为28*28的形式
    x_test = np.reshape(x_test_save, (len(x_test_save), 28, 28))     # 同上
else:  # 若数据集不存在，调用函数进行制作
    print('-------------Generate Datasets-----------------')
    x_train, y_train = generateds(train_path, train_txt)
    x_test, y_test = generateds(test_path, test_txt)
    print('-------------Save Datasets-----------------')
    x_train_save = np.reshape(x_train, (len(x_train), -1))
    x_test_save = np.reshape(x_test, (len(x_test), -1))
    np.save(x_train_savepath, x_train_save)
    np.save(y_train_savepath, y_train)
    np.save(x_test_savepath, x_test_save)
    np.save(y_test_savepath, y_test)

# 第三步，搭建网络结构
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
# 第四步，配置训练方法
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])
# 第五步，执行训练，依次为训练集样本，训练集标签，小批量大小32，训练轮次5，测试集，训练集循环1轮次进行一次测试
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), validation_freq=1)
# 第六步，打印网络结构和参数统计
model.summary()

为了方便后续识别使用，本项目依然使用从tensorflow库直接导入的MNIST的数据集训练模型。